<a href="https://colab.research.google.com/github/sinaseyfi73/ECF_homework_SinaSeyfi/blob/main/ECF_hm3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Is the relationship between equity issuance and market liquidity just a matter of region?

This code has been written by Anh Nguyen and Sina Seyfi, for the course Empirical Corporate Finance 2021, assignment 3. We replicate results from the Paper Hanselaar, Rogier M, Rene M Stulz, and Mathijs A Van Dijk, 2019, Do firms issue more equity when markets become more liquid?, *Journal of Financial Economics* 133, 64–82. 

If there are any questions contact authors.


Anh Nguyen ([Email:anh.thi.nguyen@hanken.fi](mailto:anh.thi.nguyen@hanken.fi)) is a Finance Ph.D. student in Hanken School of Economics.

Sina Seyfi ([Email:sina.seyfi@aalto.fi](mailto:sina.seyfi@aalto.fi)) is a Finance Ph.D. student from Aalto University, School of Business. 



# Introduction

In [ ]:
import numpy as np
import pandas as pd
import stat
import matplotlib.pyplot as plt
!pip install linearmodels

# Perform PooledOLS
from linearmodels import PooledOLS
import statsmodels.api as sm
from linearmodels import PanelOLS


In this part, we read the data. In order to be able to run the codes afterward, you need to change tha path of the data to the excel file that we provided.

# Table 2 - Replication

We define the Issues data, Liquidity and returns:

In [ ]:
path = '/content/drive/MyDrive/PhD/ECF/Hanselaar_Stulz_vanDijk_data/Hanselaar_Stulz_vanDijk_data.xlsx'
Issues = pd.read_excel(path, sheet_name='Issues', skiprows=5)
Issues = Issues.set_index(['YearQuarter']) 
Issues = Issues.diff()
Issues_1 = Issues.shift()
Issues_2 = Issues.shift(2)
Issues_3 = Issues.shift(3)
 
Liquidity = pd.read_excel(path, sheet_name='Liquidity', skiprows=5)
Liquidity = Liquidity.set_index(['YearQuarter'])  
Liquidity = Liquidity.diff()
Liquidity_2p = Liquidity.shift(-2)
Liquidity_1p = Liquidity.shift(-1)
Liquidity_1 = Liquidity.shift(1)
Liquidity_2 = Liquidity.shift(2)
Liquidity_3 = Liquidity.shift(3)
Liquidity_4 = Liquidity.shift(4)
Liquidity_5 = Liquidity.shift(5)
Liquidity_6 = Liquidity.shift(6)

 
Market_returns = pd.read_excel(path, sheet_name='Market returns', skiprows=5)
Market_returns = Market_returns.set_index(['YearQuarter'])  

Market_returns_1p = Market_returns.shift(-1)
Market_returns_1 = Market_returns.shift(1)
Market_returns_2 = Market_returns.shift(2)
Market_returns_3 = Market_returns.shift(3)
Market_returns_4 = Market_returns.shift(4)
 




names_of_variables = [Issues, 
                      Liquidity_2p, Liquidity_1p, Liquidity,
                      Liquidity_1, Liquidity_2, Liquidity_3, Liquidity_4,
                      Liquidity_5, Liquidity_6,
                      Market_returns_1p, Market_returns, Market_returns_1,
                      Market_returns_2, Market_returns_3, Market_returns_4,
                      Issues_1, Issues_2, Issues_3]

list_of_variables = ['Issues', 
                      'Liquidity_2p', 'Liquidity_1p', 'Liquidity',
                      'Liquidity_1', 'Liquidity_2', 'Liquidity_3', 'Liquidity_4',
                      'Liquidity_5', 'Liquidity_6',
                      'Market_returns_1p', 'Market_returns', 'Market_returns_1',
                      'Market_returns_2', 'Market_returns_3', 'Market_returns_4',
                      'Issues_1', 'Issues_2', 'Issues_3']

i = 0
for data in names_of_variables:
  data  = pd.DataFrame(data.stack(dropna = False ), columns=[list_of_variables[i]])
  data.index = data.index.set_names(['YearQuarter', 'Country'])
  if i == 0: new_name_of_variables = pd.DataFrame(index = data.index, columns= list_of_variables)
  new_name_of_variables[list_of_variables[i]] = data
  i += 1


panel_data = pd.concat([new_name_of_variables,
                        pd.get_dummies(pd.DataFrame(new_name_of_variables.index.get_level_values(0) % 10, 
                                                index = new_name_of_variables.index)['YearQuarter'])], axis = 1)



panel_data                                                

Issues  Liquidity_2p  Liquidity_1p  ...  2  3  4
YearQuarter Country                                          ...         
19901       ARG             NaN           NaN           NaN  ...  0  0  0
            AUS             NaN           NaN           NaN  ...  0  0  0
            AUT             NaN           NaN           NaN  ...  0  0  0
            BEL             NaN           NaN           NaN  ...  0  0  0
            BRA             NaN           NaN           NaN  ...  0  0  0
...                         ...           ...           ...  ... .. .. ..
20144       SWE        1.498864           NaN           NaN  ...  0  0  1
            THA        1.521491           NaN           NaN  ...  0  0  1
            USANASDAQ  1.481866           NaN           NaN  ...  0  0  1
            USANYSE    0.079183           NaN           NaN  ...  0  0  1
            ZAF        2.606112           NaN           NaN  ...  0  0  1

[4000 rows x 23 columns]

In [ ]:
panel_data = panel_data.reset_index().set_index(['Country', 'YearQuarter'])

panel_data

,,Issues,Liquidity_2p,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Liquidity_5,Liquidity_6,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,1.498864,NaN,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,-2.506717,1.779274,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,1.521491,NaN,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,-3.714794,-2.273362,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,1.481866,NaN,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,-0.070041,-0.254677,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,-0.281044,-1.890656,1.379668,0,0,0,1


Summary Statistic

In [ ]:
panel_data.dropna()[['Issues', 'Liquidity', 'Liquidity_1',
                            'Market_returns', 'Market_returns_1', 'Issues_1']].corr()

,Issues,Liquidity,Liquidity_1,Market_returns,Market_returns_1,Issues_1
Issues,1.000000,0.116113,-0.015939,0.098701,-0.031545,-0.661083
Liquidity,0.116113,1.000000,-0.547862,0.178499,-0.203004,-0.103994
Liquidity_1,-0.015939,-0.547862,1.000000,-0.000594,0.176800,0.113211
Market_returns,0.098701,0.178499,-0.000594,1.000000,0.047959,-0.051111
Market_returns_1,-0.031545,-0.203004,0.176800,0.047959,1.000000,0.100155
Issues_1,-0.661083,-0.103994,0.113211,-0.051111,0.100155,1.000000


Panel regressions

## table 2 model 1

In [ ]:
model = PanelOLS(panel_data.dropna().Issues,
                 panel_data.dropna().drop(columns=['Issues', 'Liquidity_2p',
                                                   'Liquidity_5', 'Liquidity_6', 'Issues_2', 'Issues_3']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.4747
Estimator:                   PanelOLS   R-squared (Between):             -0.3087
No. Observations:                3028   R-squared (Within):               0.4748
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.4747
Time:                        12:13:50   Log-likelihood                   -4943.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      170.06
Entities:                          40   P-value                           0.0000
Avg Obs:                       75.700   Distribution:                 F(16,3011)
Min Obs:                       0.0000                                           
Max Obs:                       89.000   F-statistic (robust):             169.09
                            

In [ ]:
model = PanelOLS(panel_data.fillna(0).Issues,
                 panel_data.fillna(0).drop(columns=['Issues', 'Liquidity_2p',
                                                   'Liquidity_5', 'Liquidity_6', 'Issues_2', 'Issues_3']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.4558
Estimator:                   PanelOLS   R-squared (Between):             -0.7399
No. Observations:                4000   R-squared (Within):               0.4558
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.4558
Time:                        12:13:50   Log-likelihood                   -6209.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      208.49
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(16,3983)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             208.43
                            

## table 2 model 2

In [ ]:
model = PanelOLS(panel_data.dropna().Issues,
                 panel_data.dropna().drop(columns=['Issues', 'Liquidity_2p',
                                                   'Liquidity_5', 'Liquidity_6']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6426
Estimator:                   PanelOLS   R-squared (Between):             -0.2467
No. Observations:                3028   R-squared (Within):               0.6427
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6426
Time:                        12:13:50   Log-likelihood                   -4360.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      300.56
Entities:                          40   P-value                           0.0000
Avg Obs:                       75.700   Distribution:                 F(18,3009)
Min Obs:                       0.0000                                           
Max Obs:                       89.000   F-statistic (robust):             299.30
                            

In [ ]:
model = PanelOLS(panel_data.fillna(0).Issues,
                 panel_data.fillna(0).drop(columns=['Issues', 'Liquidity_2p',
                                                   'Liquidity_5', 'Liquidity_6']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6273
Estimator:                   PanelOLS   R-squared (Between):             -4.6129
No. Observations:                4000   R-squared (Within):               0.6276
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6273
Time:                        12:13:50   Log-likelihood                   -5451.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      372.31
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(18,3981)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             372.22
                            

## table 2 model 3

In [ ]:
model = PanelOLS(panel_data.dropna().Issues,
                 panel_data.dropna().drop(columns=['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6432
Estimator:                   PanelOLS   R-squared (Between):             -0.2894
No. Observations:                3028   R-squared (Within):               0.6433
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6432
Time:                        12:13:51   Log-likelihood                   -4357.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      258.06
Entities:                          40   P-value                           0.0000
Avg Obs:                       75.700   Distribution:                 F(21,3006)
Min Obs:                       0.0000                                           
Max Obs:                       89.000   F-statistic (robust):             256.98
                            

In [ ]:
model = PanelOLS(panel_data.fillna(0).Issues,
                 panel_data.fillna(0).drop(columns=['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6276
Estimator:                   PanelOLS   R-squared (Between):             -4.9267
No. Observations:                4000   R-squared (Within):               0.6278
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6276
Time:                        12:13:51   Log-likelihood                   -5450.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      319.22
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(21,3978)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             319.15
                            

## table 2 model 4


In [ ]:
model = PanelOLS(panel_data.dropna().Issues,
                 panel_data.dropna().drop(columns=['Issues', 'Liquidity_2p',
                                                   'Liquidity_1p', 'Liquidity_4',
                                                   'Liquidity_5','Liquidity_6', 'Market_returns_1p']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6419
Estimator:                   PanelOLS   R-squared (Between):             -0.2519
No. Observations:                3028   R-squared (Within):               0.6420
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6419
Time:                        12:13:51   Log-likelihood                   -4363.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      359.98
Entities:                          40   P-value                           0.0000
Avg Obs:                       75.700   Distribution:                 F(15,3012)
Min Obs:                       0.0000                                           
Max Obs:                       89.000   F-statistic (robust):             354.52
                            

In [ ]:
model = PanelOLS(panel_data.fillna(0).Issues,
                 panel_data.fillna(0).drop(columns=['Issues', 'Liquidity_2p',
                                                   'Liquidity_1p', 'Liquidity_4',
                                                   'Liquidity_5','Liquidity_6', 'Market_returns_1p']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6266
Estimator:                   PanelOLS   R-squared (Between):             -4.5186
No. Observations:                4000   R-squared (Within):               0.6269
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6266
Time:                        12:13:51   Log-likelihood                   -5455.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      445.78
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(15,3984)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             445.67
                            

## table 2 model 6


In [ ]:
model = PanelOLS(panel_data.dropna().Issues,
                 panel_data.dropna().drop(columns=['Issues', 'Liquidity_2p',
                                                   'Liquidity_5','Liquidity_6', 'Market_returns',
                                                   'Market_returns_1p', 'Market_returns_1', 'Market_returns_2',
                                                   'Market_returns_3', 'Market_returns_4']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6357
Estimator:                   PanelOLS   R-squared (Between):             -0.3229
No. Observations:                3028   R-squared (Within):               0.6358
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6357
Time:                        12:13:51   Log-likelihood                   -4389.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      438.48
Entities:                          40   P-value                           0.0000
Avg Obs:                       75.700   Distribution:                 F(12,3015)
Min Obs:                       0.0000                                           
Max Obs:                       89.000   F-statistic (robust):             424.21
                            

In [ ]:
model = PanelOLS(panel_data.fillna(0).Issues,
                 panel_data.fillna(0).drop(columns=['Issues', 'Liquidity_2p',
                                                   'Liquidity_5','Liquidity_6', 'Market_returns',
                                                   'Market_returns_1p', 'Market_returns_1', 'Market_returns_2',
                                                   'Market_returns_3', 'Market_returns_4']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6220
Estimator:                   PanelOLS   R-squared (Between):             -4.7676
No. Observations:                4000   R-squared (Within):               0.6223
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6220
Time:                        12:13:52   Log-likelihood                   -5480.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      546.78
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(12,3987)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             544.77
                            

## table 2 model 7


In [ ]:
model = PanelOLS(panel_data.dropna().Issues,
                 panel_data.dropna().drop(columns=['Issues', 'Liquidity_2p', 'Liquidity_1p', 'Liquidity',
                                                   'Liquidity_1', 'Liquidity_2', 'Liquidity_3', 'Liquidity_4', 
                                                   'Liquidity_5','Liquidity_6']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6371
Estimator:                   PanelOLS   R-squared (Between):             -0.3072
No. Observations:                3028   R-squared (Within):               0.6372
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6371
Time:                        12:13:52   Log-likelihood                   -4383.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      441.05
Entities:                          40   P-value                           0.0000
Avg Obs:                       75.700   Distribution:                 F(12,3015)
Min Obs:                       0.0000                                           
Max Obs:                       89.000   F-statistic (robust):             439.19
                            

In [ ]:
model = PanelOLS(panel_data.fillna(0).Issues,
                 panel_data.fillna(0).drop(columns=['Issues', 'Liquidity_2p', 'Liquidity_1p', 'Liquidity',
                                                   'Liquidity_1', 'Liquidity_2', 'Liquidity_3', 'Liquidity_4', 
                                                   'Liquidity_5','Liquidity_6']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6213
Estimator:                   PanelOLS   R-squared (Between):             -4.2910
No. Observations:                4000   R-squared (Within):               0.6215
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6213
Time:                        12:13:52   Log-likelihood                   -5484.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      545.05
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(12,3987)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             544.93
                            

# Table 3 - replication

Again we need to define our data.

In [ ]:
# Defining Volatility, and its lags
Volatility = pd.read_excel(path, sheet_name='Volatility', skiprows=5)
Volatility = Volatility.set_index(['YearQuarter'])
Volatility = Volatility.diff()
Volatility_1p = Volatility.shift(-1)
Volatility_1 = Volatility.shift(1)
Volatility_2 = Volatility.shift(2)
Volatility_3 = Volatility.shift(3)
Volatility_4 = Volatility.shift(4)

v = [Volatility_1p, Volatility, Volatility_1,
             Volatility_2, Volatility_3, Volatility_4]
names = ['Volatility_1p', 'Volatility', 'Volatility_1',
             'Volatility_2', 'Volatility_3', 'Volatility_4']             
i = 0             
for data in v: 
    v[i] = pd.DataFrame(data.stack(dropna = False ), columns= [names[i]])
    v[i].index = v[i].index.set_names(['YearQuarter', 'Country'])
    v[i] = v[i].reset_index().set_index(['Country', 'YearQuarter'])
    i += 1
[Volatility_1p, Volatility, Volatility_1,
             Volatility_2, Volatility_3, Volatility_4] = v




# Defining Turnover, and its lags
Turnover = pd.read_excel(path, sheet_name='Turnover', skiprows=5)
Turnover = Turnover.set_index(['YearQuarter'])
Turnover = Turnover.diff()
Turnover_1p = Turnover.shift(-1)
Turnover_1 = Turnover.shift(1)
Turnover_2 = Turnover.shift(2)
Turnover_3 = Turnover.shift(3)
Turnover_4 = Turnover.shift(4)

v = [Turnover_1p, Turnover, Turnover_1,
             Turnover_2, Turnover_3, Turnover_4]
names = ['Turnover_1p', 'Turnover', 'Turnover_1',
             'Turnover_2', 'Turnover_3', 'Turnover_4']             
i = 0             
for data in v: 
    v[i] = pd.DataFrame(data.stack(dropna = False ), columns= [names[i]])
    v[i].index = v[i].index.set_names(['YearQuarter', 'Country'])
    v[i] = v[i].reset_index().set_index(['Country', 'YearQuarter'])
    i += 1
[Turnover_1p, Turnover, Turnover_1,
             Turnover_2, Turnover_3, Turnover_4] = v


# Defining Liquidity_risk, and its lags
Liquidity_risk = pd.read_excel(path, sheet_name='Liquidity risk', skiprows=5)
Liquidity_risk = Liquidity_risk.set_index(['YearQuarter'])
Liquidity_risk = Liquidity_risk.diff()
Liquidity_risk_1p = Liquidity_risk.shift(-1)
Liquidity_risk_1 = Liquidity_risk.shift(1)
Liquidity_risk_2 = Liquidity_risk.shift(2)
Liquidity_risk_3 = Liquidity_risk.shift(3)
Liquidity_risk_4 = Liquidity_risk.shift(4)

v = [Liquidity_risk_1p, Liquidity_risk, Liquidity_risk_1,
             Liquidity_risk_2, Liquidity_risk_3, Liquidity_risk_4]
names = ['Liquidity_risk_1p', 'Liquidity_risk', 'Liquidity_risk_1',
             'Liquidity_risk_2', 'Liquidity_risk_3', 'Liquidity_risk_4']             
i = 0             
for data in v: 
    v[i] = pd.DataFrame(data.stack(dropna = False ), columns= [names[i]])
    v[i].index = v[i].index.set_names(['YearQuarter', 'Country'])
    v[i] = v[i].reset_index().set_index(['Country', 'YearQuarter'])
    i += 1
[Liquidity_risk_1p, Liquidity_risk, Liquidity_risk_1,
             Liquidity_risk_2, Liquidity_risk_3, Liquidity_risk_4] = v




# Defining MB_ratio, and its lags
MB_ratio = pd.read_excel(path, sheet_name='Market-to-book ratio', skiprows=5)
MB_ratio = MB_ratio.set_index(['YearQuarter'])
MB_ratio = MB_ratio.diff()
MB_ratio_1p = MB_ratio.shift(-1)
MB_ratio_1 = MB_ratio.shift(1)
MB_ratio_2 = MB_ratio.shift(2)
MB_ratio_3 = MB_ratio.shift(3)
MB_ratio_4 = MB_ratio.shift(4)

v = [MB_ratio_1p, MB_ratio, MB_ratio_1,
             MB_ratio_2, MB_ratio_3, MB_ratio_4]
names = ['MB_ratio_1p', 'MB_ratio', 'MB_ratio_1',
             'MB_ratio_2', 'MB_ratio_3', 'MB_ratio_4']             
i = 0             
for data in v: 
    v[i] = pd.DataFrame(data.stack(dropna = False ), columns= [names[i]])
    v[i].index = v[i].index.set_names(['YearQuarter', 'Country'])
    v[i] = v[i].reset_index().set_index(['Country', 'YearQuarter'])
    i += 1
[MB_ratio_1p, MB_ratio, MB_ratio_1,
             MB_ratio_2, MB_ratio_3, MB_ratio_4] = v


# Defining price_earning, and its lags
price_earning = pd.read_excel(path, sheet_name='Price earnings ratio', skiprows=5)
price_earning = price_earning.set_index(['YearQuarter'])
price_earning = price_earning.diff()
price_earning_1p = price_earning.shift(-1)
price_earning_1 = price_earning.shift(1)
price_earning_2 = price_earning.shift(2)
price_earning_3 = price_earning.shift(3)
price_earning_4 = price_earning.shift(4)

v = [price_earning_1p, price_earning, price_earning_1,
             price_earning_2, price_earning_3, price_earning_4]
names = ['price_earning_1p', 'price_earning', 'price_earning_1',
             'price_earning_2', 'price_earning_3', 'price_earning_4']             
i = 0             
for data in v: 
    v[i] = pd.DataFrame(data.stack(dropna = False ), columns= [names[i]])
    v[i].index = v[i].index.set_names(['YearQuarter', 'Country'])
    v[i] = v[i].reset_index().set_index(['Country', 'YearQuarter'])
    i += 1
[price_earning_1p, price_earning, price_earning_1,
             price_earning_2, price_earning_3, price_earning_4] = v


# Defining Dividend_yield, and its lags
Dividend_yield = pd.read_excel(path, sheet_name='Dividend yield', skiprows=5)
Dividend_yield = Dividend_yield.set_index(['YearQuarter'])
Dividend_yield = Dividend_yield.diff()
Dividend_yield_1p = Dividend_yield.shift(-1)
Dividend_yield_1 = Dividend_yield.shift(1)
Dividend_yield_2 = Dividend_yield.shift(2)
Dividend_yield_3 = Dividend_yield.shift(3)
Dividend_yield_4 = Dividend_yield.shift(4)

v = [Dividend_yield_1p, Dividend_yield, Dividend_yield_1,
             Dividend_yield_2, Dividend_yield_3, Dividend_yield_4]
names = ['Dividend_yield_1p', 'Dividend_yield', 'Dividend_yield_1',
             'Dividend_yield_2', 'Dividend_yield_3', 'Dividend_yield_4']             
i = 0             
for data in v: 
    v[i] = pd.DataFrame(data.stack(dropna = False ), columns= [names[i]])
    v[i].index = v[i].index.set_names(['YearQuarter', 'Country'])
    v[i] = v[i].reset_index().set_index(['Country', 'YearQuarter'])
    i += 1
[Dividend_yield_1p, Dividend_yield, Dividend_yield_1,
             Dividend_yield_2, Dividend_yield_3, Dividend_yield_4] = v

Dividend_yield

,,Dividend_yield
Country,YearQuarter,
ARG,19901,NaN
AUS,19901,NaN
AUT,19901,NaN
BEL,19901,NaN
BRA,19901,NaN
...,...,...
SWE,20144,0.253131
THA,20144,-0.012640
USANASDAQ,20144,NaN


## Table 3 model 1

In [ ]:
panel3 = panel_data[['Issues', 'Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3

,,Issues,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,1.498864,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,1.521491,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,1.481866,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna().Issues,
                 panel3.dropna().drop(columns = ['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6421
Estimator:                   PanelOLS   R-squared (Between):              0.3458
No. Observations:                3087   R-squared (Within):               0.6421
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6421
Time:                        12:13:56   Log-likelihood                   -4438.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      305.74
Entities:                          40   P-value                           0.0000
Avg Obs:                       77.175   Distribution:                 F(18,3068)
Min Obs:                       0.0000                                           
Max Obs:                       92.000   F-statistic (robust):             304.54
                            

In [ ]:
model = PanelOLS(panel3.fillna(0).Issues,
                 panel3.fillna(0).drop(columns = ['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6273
Estimator:                   PanelOLS   R-squared (Between):             -4.6129
No. Observations:                4000   R-squared (Within):               0.6276
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6273
Time:                        12:13:57   Log-likelihood                   -5451.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      372.31
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(18,3981)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             372.22
                            

## Table 3 model 2

In [ ]:
panel3 = panel_data[['Issues', 'Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3 = pd.concat([panel3.drop(columns= ['Issues_1',	'Issues_2',
                                          'Issues_3' ,1, 2, 3, 4]),
                    Volatility_1p, Volatility, Volatility_1,
             Volatility_2, Volatility_3, Volatility_4,
                    panel3[['Issues_1',	'Issues_2',
                           'Issues_3' ,1, 2, 3, 4]]], axis = 1)

panel3 

,,Issues,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,Volatility_1p,Volatility,Volatility_1,Volatility_2,Volatility_3,Volatility_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,1.498864,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,NaN,0.551890,0.592627,-0.582295,0.276356,0.607144,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,1.521491,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,NaN,0.661549,0.453656,-0.861338,1.112191,-0.952689,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,1.481866,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,NaN,0.849512,-0.184314,-0.379695,0.133448,0.679250,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna().Issues,
                 panel3.dropna().drop(columns=['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6436
Estimator:                   PanelOLS   R-squared (Between):              0.3594
No. Observations:                3087   R-squared (Within):               0.6436
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6436
Time:                        12:13:57   Log-likelihood                   -4432.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      230.40
Entities:                          40   P-value                           0.0000
Avg Obs:                       77.175   Distribution:                 F(24,3062)
Min Obs:                       0.0000                                           
Max Obs:                       92.000   F-statistic (robust):             229.49
                            

In [ ]:
model = PanelOLS(panel3.fillna(0).Issues,
                 panel3.fillna(0).drop(columns=['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6290
Estimator:                   PanelOLS   R-squared (Between):             -4.7415
No. Observations:                4000   R-squared (Within):               0.6293
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6290
Time:                        12:13:57   Log-likelihood                   -5442.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      280.85
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(24,3975)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             280.79
                            

## Table 3 model 3

In [ ]:
panel3 = panel_data[['Issues', 'Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3 = pd.concat([panel3.drop(columns= ['Issues_1',	'Issues_2',
                                          'Issues_3' ,1, 2, 3, 4]),
                    Turnover_1p, Turnover, Turnover_1,
                    Turnover_2, Turnover_3, Turnover_4,
                    panel3[['Issues_1',	'Issues_2',
                           'Issues_3' ,1, 2, 3, 4]]], axis = 1)

panel3 

,,Issues,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,Turnover_1p,Turnover,Turnover_1,Turnover_2,Turnover_3,Turnover_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,1.498864,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,NaN,1.727244,1.460540,-2.151656,-0.174647,2.011237,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,1.521491,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,NaN,-0.988385,-0.622032,-0.451663,0.952667,-0.037150,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,1.481866,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,NaN,2.786400,0.633407,-2.708608,-0.413009,2.304989,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna().Issues,
                 panel3.dropna().drop(columns=['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6441
Estimator:                   PanelOLS   R-squared (Between):              0.3388
No. Observations:                3087   R-squared (Within):               0.6442
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6441
Time:                        12:13:58   Log-likelihood                   -4429.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      230.93
Entities:                          40   P-value                           0.0000
Avg Obs:                       77.175   Distribution:                 F(24,3062)
Min Obs:                       0.0000                                           
Max Obs:                       92.000   F-statistic (robust):             230.02
                            

In [ ]:
model = PanelOLS(panel3.fillna(0).Issues,
                 panel3.fillna(0).drop(columns=['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6292
Estimator:                   PanelOLS   R-squared (Between):             -4.9933
No. Observations:                4000   R-squared (Within):               0.6294
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6292
Time:                        12:13:58   Log-likelihood                   -5441.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      281.01
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(24,3975)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             280.95
                            

## Table 3 model 4

In [ ]:
panel3 = panel_data[['Issues', 'Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3 = pd.concat([panel3.drop(columns= ['Issues_1',	'Issues_2',
                                          'Issues_3' ,1, 2, 3, 4]),
                    Liquidity_risk_1p, Liquidity_risk, Liquidity_risk_1,
                    Liquidity_risk_2, Liquidity_risk_3, Liquidity_risk_4,
                    panel3[['Issues_1',	'Issues_2',
                           'Issues_3' ,1, 2, 3, 4]]], axis = 1)

panel3 

,,Issues,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,Liquidity_risk_1p,Liquidity_risk,Liquidity_risk_1,Liquidity_risk_2,Liquidity_risk_3,Liquidity_risk_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,1.498864,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,NaN,0.113444,-0.494787,0.382793,0.097409,-0.433897,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,1.521491,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,NaN,-0.342368,0.262530,0.574273,-1.543209,1.296697,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,1.481866,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,NaN,-0.001504,0.001498,0.000134,0.000148,0.000704,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna().Issues,
                 panel3.dropna().drop(columns=['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6446
Estimator:                   PanelOLS   R-squared (Between):              0.1357
No. Observations:                3079   R-squared (Within):               0.6447
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6446
Time:                        12:13:58   Log-likelihood                   -4419.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      230.83
Entities:                          40   P-value                           0.0000
Avg Obs:                       76.975   Distribution:                 F(24,3054)
Min Obs:                       0.0000                                           
Max Obs:                       92.000   F-statistic (robust):             230.74
                            

In [ ]:
model = PanelOLS(panel3.fillna(0).Issues,
                 panel3.fillna(0).drop(columns=['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6281
Estimator:                   PanelOLS   R-squared (Between):             -4.6245
No. Observations:                4000   R-squared (Within):               0.6283
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6281
Time:                        12:13:58   Log-likelihood                   -5447.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      279.68
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(24,3975)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             279.66
                            

## Table 3 model 5

In [ ]:
panel3 = panel_data[['Issues', 'Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3 = pd.concat([panel3.drop(columns= ['Issues_1',	'Issues_2',
                                          'Issues_3' ,1, 2, 3, 4]),
                    MB_ratio_1p, MB_ratio, MB_ratio_1,
                    MB_ratio_2, MB_ratio_3, MB_ratio_4,
                    panel3[['Issues_1',	'Issues_2',
                           'Issues_3' ,1, 2, 3, 4]]], axis = 1)

panel3 

,,Issues,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,MB_ratio_1p,MB_ratio,MB_ratio_1,MB_ratio_2,MB_ratio_3,MB_ratio_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,1.498864,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,NaN,-0.417895,-0.069731,-1.183496,0.607923,1.115517,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,1.521491,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,NaN,-0.058960,-0.024731,1.663947,-0.842180,0.354244,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,1.481866,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,NaN,NaN,NaN,NaN,NaN,NaN,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna().Issues,
                 panel3.dropna().drop(columns=['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6397
Estimator:                   PanelOLS   R-squared (Between):              0.4357
No. Observations:                2901   R-squared (Within):               0.6397
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6397
Time:                        12:13:59   Log-likelihood                   -4188.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      212.78
Entities:                          40   P-value                           0.0000
Avg Obs:                       72.525   Distribution:                 F(24,2876)
Min Obs:                       0.0000                                           
Max Obs:                       92.000   F-statistic (robust):             212.20
                            

In [ ]:
model = PanelOLS(panel3.fillna(0).Issues,
                 panel3.fillna(0).drop(columns=['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6283
Estimator:                   PanelOLS   R-squared (Between):             -4.7717
No. Observations:                4000   R-squared (Within):               0.6285
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6283
Time:                        12:13:59   Log-likelihood                   -5446.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      279.91
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(24,3975)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             279.85
                            

## Table 3 model 6

In [ ]:
panel3 = panel_data[['Issues', 'Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3 = pd.concat([panel3.drop(columns= ['Issues_1',	'Issues_2',
                                          'Issues_3' ,1, 2, 3, 4]),
                    price_earning_1p, price_earning, price_earning_1,
                    price_earning_2, price_earning_3, price_earning_4,
                    panel3[['Issues_1',	'Issues_2',
                           'Issues_3' ,1, 2, 3, 4]]], axis = 1)

panel3 

,,Issues,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,price_earning_1p,price_earning,price_earning_1,price_earning_2,price_earning_3,price_earning_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,1.498864,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,NaN,-0.801001,0.255672,-0.091201,-1.237548,1.757488,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,1.521491,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,NaN,-0.092191,0.506331,0.494441,-0.320913,0.444449,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,1.481866,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,NaN,NaN,NaN,NaN,NaN,NaN,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna().Issues,
                 panel3.dropna().drop(columns=['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6393
Estimator:                   PanelOLS   R-squared (Between):              0.4358
No. Observations:                2903   R-squared (Within):               0.6393
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6393
Time:                        12:13:59   Log-likelihood                   -4191.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      212.55
Entities:                          40   P-value                           0.0000
Avg Obs:                       72.575   Distribution:                 F(24,2878)
Min Obs:                       0.0000                                           
Max Obs:                       92.000   F-statistic (robust):             211.98
                            

In [ ]:
model = PanelOLS(panel3.fillna(0).Issues,
                 panel3.fillna(0).drop(columns=['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6280
Estimator:                   PanelOLS   R-squared (Between):             -4.4811
No. Observations:                4000   R-squared (Within):               0.6282
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6280
Time:                        12:13:59   Log-likelihood                   -5448.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      279.61
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(24,3975)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             279.55
                            

## Table 3 model 7

In [ ]:
panel3 = panel_data[['Issues', 'Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3 = pd.concat([panel3.drop(columns= ['Issues_1',	'Issues_2',
                                          'Issues_3' ,1, 2, 3, 4]),
                    Dividend_yield_1p, Dividend_yield, Dividend_yield_1,
                    Dividend_yield_2, Dividend_yield_3, Dividend_yield_4,
                    panel3[['Issues_1',	'Issues_2',
                           'Issues_3' ,1, 2, 3, 4]]], axis = 1)

panel3 

,,Issues,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,Dividend_yield_1p,Dividend_yield,Dividend_yield_1,Dividend_yield_2,Dividend_yield_3,Dividend_yield_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,1.498864,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,NaN,0.253131,-0.607968,1.079657,0.090068,-0.944078,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,1.521491,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,NaN,-0.012640,0.175672,-1.014638,0.338181,-0.079026,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,1.481866,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,NaN,NaN,NaN,NaN,NaN,NaN,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna().Issues,
                 panel3.dropna().drop(columns=['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6394
Estimator:                   PanelOLS   R-squared (Between):              0.4409
No. Observations:                2903   R-squared (Within):               0.6394
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6394
Time:                        12:14:00   Log-likelihood                   -4191.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      212.63
Entities:                          40   P-value                           0.0000
Avg Obs:                       72.575   Distribution:                 F(24,2878)
Min Obs:                       0.0000                                           
Max Obs:                       92.000   F-statistic (robust):             212.06
                            

In [ ]:
model = PanelOLS(panel3.fillna(0).Issues,
                 panel3.fillna(0).drop(columns=['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6276
Estimator:                   PanelOLS   R-squared (Between):             -4.7133
No. Observations:                4000   R-squared (Within):               0.6279
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6276
Time:                        12:14:00   Log-likelihood                   -5450.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      279.14
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(24,3975)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             279.08
                            

## Table 3 model 8

In [ ]:
panel3 = panel_data[['Issues', 'Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3 = pd.concat([panel3.drop(columns= ['Issues_1',	'Issues_2',
                                          'Issues_3' ,1, 2, 3, 4]),
                    Volatility_1p, Volatility, Volatility_1,
                    Volatility_2, Volatility_3, Volatility_4,
                    Turnover_1p, Turnover, Turnover_1,
                    Turnover_2, Turnover_3, Turnover_4,
                    Liquidity_risk_1p, Liquidity_risk, Liquidity_risk_1,
                    Liquidity_risk_2, Liquidity_risk_3, Liquidity_risk_4,
                    MB_ratio_1p, MB_ratio, MB_ratio_1,
                    MB_ratio_2, MB_ratio_3, MB_ratio_4,
                    price_earning_1p, price_earning, price_earning_1,
                    price_earning_2, price_earning_3, price_earning_4,
                    Dividend_yield_1p, Dividend_yield, Dividend_yield_1,
                    Dividend_yield_2, Dividend_yield_3, Dividend_yield_4,
                    panel3[['Issues_1',	'Issues_2',
                           'Issues_3' ,1, 2, 3, 4]]], axis = 1)

panel3 

,,Issues,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,Volatility_1p,Volatility,Volatility_1,Volatility_2,Volatility_3,Volatility_4,Turnover_1p,Turnover,Turnover_1,Turnover_2,Turnover_3,Turnover_4,Liquidity_risk_1p,Liquidity_risk,Liquidity_risk_1,Liquidity_risk_2,Liquidity_risk_3,Liquidity_risk_4,MB_ratio_1p,MB_ratio,MB_ratio_1,MB_ratio_2,MB_ratio_3,MB_ratio_4,price_earning_1p,price_earning,price_earning_1,price_earning_2,price_earning_3,price_earning_4,Dividend_yield_1p,Dividend_yield,Dividend_yield_1,Dividend_yield_2,Dividend_yield_3,Dividend_yield_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,1.498864,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,NaN,0.551890,0.592627,-0.582295,0.276356,0.607144,NaN,1.727244,1.460540,-2.151656,-0.174647,2.011237,NaN,0.113444,-0.494787,0.382793,0.097409,-0.433897,NaN,-0.417895,-0.069731,-1.183496,0.607923,1.115517,NaN,-0.801001,0.255672,-0.091201,-1.237548,1.757488,NaN,0.253131,-0.607968,1.079657,0.090068,-0.944078,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,1.521491,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,NaN,0.661549,0.453656,-0.861338,1.112191,-0.952689,NaN,-0.988385,-0.622032,-0.451663,0.952667,-0.037150,NaN,-0.342368,0.262530,0.574273,-1.543209,1.296697,NaN,-0.058960,-0.024731,1.663947,-0.842180,0.354244,NaN,-0.092191,0.506331,0.494441,-0.320913,0.444449,NaN,-0.012640,0.175672,-1.014638,0.338181,-0.079026,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,1.481866,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,NaN,0.849512,-0.184314,-0.379695,0.133448,0.679250,NaN,2.786400,0.633407,-2.708608,-0.413009,2.304989,NaN,-0.001504,0.001498,0.000134,0.000148,0.000704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna().Issues,
                 panel3.dropna().drop(columns=['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6486
Estimator:                   PanelOLS   R-squared (Between):              0.2804
No. Observations:                2894   R-squared (Within):               0.6487
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6486
Time:                        12:14:00   Log-likelihood                   -4144.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      97.047
Entities:                          40   P-value                           0.0000
Avg Obs:                       72.350   Distribution:                 F(54,2839)
Min Obs:                       0.0000                                           
Max Obs:                       92.000   F-statistic (robust):             96.978
                            

In [ ]:
model = PanelOLS(panel3.fillna(0).Issues,
                 panel3.fillna(0).drop(columns=['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6343
Estimator:                   PanelOLS   R-squared (Between):             -5.0211
No. Observations:                4000   R-squared (Within):               0.6345
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6343
Time:                        12:14:00   Log-likelihood                   -5414.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      126.70
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(54,3945)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             126.69
                            

# Table 4

Defining variables and data

In [ ]:
# Defining GDP_growth, and its lags
GDP_growth = pd.read_excel(path, sheet_name='GDP growth', skiprows=5)
GDP_growth = GDP_growth.set_index(['YearQuarter'])
#GDP_growth = GDP_growth.diff()
GDP_growth_1p = GDP_growth.shift(-1)
GDP_growth_1 = GDP_growth.shift(1)
GDP_growth_2 = GDP_growth.shift(2)
GDP_growth_3 = GDP_growth.shift(3)
GDP_growth_4 = GDP_growth.shift(4)

v = [GDP_growth_1p, GDP_growth, GDP_growth_1,
             GDP_growth_2, GDP_growth_3, GDP_growth_4]
names = ['GDP_growth_1p', 'GDP_growth', 'GDP_growth_1',
             'GDP_growth_2', 'GDP_growth_3', 'GDP_growth_4']             
i = 0             
for data in v: 
    v[i] = pd.DataFrame(data.stack(dropna = False ), columns= [names[i]])
    v[i].index = v[i].index.set_names(['YearQuarter', 'Country'])
    v[i] = v[i].reset_index().set_index(['Country', 'YearQuarter'])
    i += 1
[GDP_growth_1p, GDP_growth, GDP_growth_1,
             GDP_growth_2, GDP_growth_3, GDP_growth_4] = v



# Defining Sales_growth, and its lags
Sales_growth = pd.read_excel(path, sheet_name='Sales growth', skiprows=5)
Sales_growth = Sales_growth.set_index(['YearQuarter'])
#Sales_growth = Sales_growth.diff()
Sales_growth_1p = Sales_growth.shift(-1)
Sales_growth_1 = Sales_growth.shift(1)
Sales_growth_2 = Sales_growth.shift(2)
Sales_growth_3 = Sales_growth.shift(3)
Sales_growth_4 = Sales_growth.shift(4)

v = [Sales_growth_1p, Sales_growth, Sales_growth_1,
             Sales_growth_2, Sales_growth_3, Sales_growth_4]
names = ['Sales_growth_1p', 'Sales_growth', 'Sales_growth_1',
             'Sales_growth_2', 'Sales_growth_3', 'Sales_growth_4']             
i = 0             
for data in v: 
    v[i] = pd.DataFrame(data.stack(dropna = False ), columns= [names[i]])
    v[i].index = v[i].index.set_names(['YearQuarter', 'Country'])
    v[i] = v[i].reset_index().set_index(['Country', 'YearQuarter'])
    i += 1
[Sales_growth_1p, Sales_growth, Sales_growth_1,
             Sales_growth_2, Sales_growth_3, Sales_growth_4] = v


# Defining LEI_growth, and its lags
LEI_growth = pd.read_excel(path, sheet_name='LEI growth', skiprows=5)
LEI_growth = LEI_growth.set_index(['YearQuarter'])
#LEI_growth = LEI_growth.diff()
LEI_growth_1p = LEI_growth.shift(-1)
LEI_growth_1 = LEI_growth.shift(1)
LEI_growth_2 = LEI_growth.shift(2)
LEI_growth_3 = LEI_growth.shift(3)
LEI_growth_4 = LEI_growth.shift(4)

v = [LEI_growth_1p, LEI_growth, LEI_growth_1,
             LEI_growth_2, LEI_growth_3, LEI_growth_4]
names = ['LEI_growth_1p', 'LEI_growth', 'LEI_growth_1',
             'LEI_growth_2', 'LEI_growth_3', 'LEI_growth_4']             
i = 0             
for data in v: 
    v[i] = pd.DataFrame(data.stack(dropna = False ), columns= [names[i]])
    v[i].index = v[i].index.set_names(['YearQuarter', 'Country'])
    v[i] = v[i].reset_index().set_index(['Country', 'YearQuarter'])
    i += 1
[LEI_growth_1p, LEI_growth, LEI_growth_1,
             LEI_growth_2, LEI_growth_3, LEI_growth_4] = v




# Defining Idiosyncratic_volatility, and its lags
Idiosyncratic_volatility = pd.read_excel(path, sheet_name='Idiosyncratic volatility', skiprows=5)
Idiosyncratic_volatility = Idiosyncratic_volatility.set_index(['YearQuarter'])
Idiosyncratic_volatility = Idiosyncratic_volatility.diff()
Idiosyncratic_volatility_1p = Idiosyncratic_volatility.shift(-1)
Idiosyncratic_volatility_1 = Idiosyncratic_volatility.shift(1)
Idiosyncratic_volatility_2 = Idiosyncratic_volatility.shift(2)
Idiosyncratic_volatility_3 = Idiosyncratic_volatility.shift(3)
Idiosyncratic_volatility_4 = Idiosyncratic_volatility.shift(4)

v = [Idiosyncratic_volatility_1p, Idiosyncratic_volatility, Idiosyncratic_volatility_1,
             Idiosyncratic_volatility_2, Idiosyncratic_volatility_3, Idiosyncratic_volatility_4]
names = ['Idiosyncratic_volatility_1p', 'Idiosyncratic_volatility', 'Idiosyncratic_volatility_1',
             'Idiosyncratic_volatility_2', 'Idiosyncratic_volatility_3', 'Idiosyncratic_volatility_4']             
i = 0             
for data in v: 
    v[i] = pd.DataFrame(data.stack(dropna = False ), columns= [names[i]])
    v[i].index = v[i].index.set_names(['YearQuarter', 'Country'])
    v[i] = v[i].reset_index().set_index(['Country', 'YearQuarter'])
    i += 1
[Idiosyncratic_volatility_1p, Idiosyncratic_volatility, Idiosyncratic_volatility_1,
             Idiosyncratic_volatility_2, Idiosyncratic_volatility_3, Idiosyncratic_volatility_4] = v



# Defining Stock_price_synchronicity, and its lags
Stock_price_synchronicity = pd.read_excel(path, sheet_name='Stock price synchronicity', skiprows=5)
Stock_price_synchronicity = Stock_price_synchronicity.set_index(['YearQuarter'])
Stock_price_synchronicity = Stock_price_synchronicity.diff()
Stock_price_synchronicity_1p = Stock_price_synchronicity.shift(-1)
Stock_price_synchronicity_1 = Stock_price_synchronicity.shift(1)
Stock_price_synchronicity_2 = Stock_price_synchronicity.shift(2)
Stock_price_synchronicity_3 = Stock_price_synchronicity.shift(3)
Stock_price_synchronicity_4 = Stock_price_synchronicity.shift(4)

v = [Stock_price_synchronicity_1p, Stock_price_synchronicity, Stock_price_synchronicity_1,
             Stock_price_synchronicity_2, Stock_price_synchronicity_3, Stock_price_synchronicity_4]
names = ['Stock_price_synchronicity_1p', 'Stock_price_synchronicity', 'Stock_price_synchronicity_1',
             'Stock_price_synchronicity_2', 'Stock_price_synchronicity_3', 'Stock_price_synchronicity_4']             
i = 0             
for data in v: 
    v[i] = pd.DataFrame(data.stack(dropna = False ), columns= [names[i]])
    v[i].index = v[i].index.set_names(['YearQuarter', 'Country'])
    v[i] = v[i].reset_index().set_index(['Country', 'YearQuarter'])
    i += 1
[Stock_price_synchronicity_1p, Stock_price_synchronicity, Stock_price_synchronicity_1,
             Stock_price_synchronicity_2, Stock_price_synchronicity_3, Stock_price_synchronicity_4] = v



## Table 4 model 1

In [ ]:
panel3 = panel_data[['Issues', 'Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3

,,Issues,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,1.498864,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,1.521491,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,1.481866,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna().Issues,
                 panel3.dropna().drop(columns = ['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6421
Estimator:                   PanelOLS   R-squared (Between):              0.3458
No. Observations:                3087   R-squared (Within):               0.6421
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6421
Time:                        12:14:04   Log-likelihood                   -4438.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      305.74
Entities:                          40   P-value                           0.0000
Avg Obs:                       77.175   Distribution:                 F(18,3068)
Min Obs:                       0.0000                                           
Max Obs:                       92.000   F-statistic (robust):             304.54
                            

In [ ]:
model = PanelOLS(panel3.fillna(0).Issues,
                 panel3.fillna(0).drop(columns = ['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6273
Estimator:                   PanelOLS   R-squared (Between):             -4.6129
No. Observations:                4000   R-squared (Within):               0.6276
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6273
Time:                        12:14:05   Log-likelihood                   -5451.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      372.31
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(18,3981)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             372.22
                            

## Table 4 model 2

In [ ]:
panel3 = panel_data[['Issues', 'Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3 = pd.concat([panel3.drop(columns= ['Issues_1',	'Issues_2',
                                          'Issues_3' ,1, 2, 3, 4]),
                    GDP_growth_1p, GDP_growth, GDP_growth_1,
             GDP_growth_2, GDP_growth_3, GDP_growth_4,
                    panel3[['Issues_1',	'Issues_2',
                           'Issues_3' ,1, 2, 3, 4]]], axis = 1)

panel3 

,,Issues,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,GDP_growth_1p,GDP_growth,GDP_growth_1,GDP_growth_2,GDP_growth_3,GDP_growth_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,-1.117565,0.258314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,1.388266,1.042828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,0.640015,1.435284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,1.498864,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,NaN,0.208137,0.136682,0.285338,-0.103249,0.113881,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,1.521491,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,NaN,NaN,NaN,NaN,NaN,NaN,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,1.481866,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,NaN,0.022434,0.261901,0.085678,-0.384667,0.010417,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna().Issues,
                 panel3.dropna().drop(columns = ['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6644
Estimator:                   PanelOLS   R-squared (Between):              0.1374
No. Observations:                2514   R-squared (Within):               0.6645
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6644
Time:                        12:14:05   Log-likelihood                   -3557.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      205.32
Entities:                          40   P-value                           0.0000
Avg Obs:                       62.850   Distribution:                 F(24,2489)
Min Obs:                       0.0000                                           
Max Obs:                       92.000   F-statistic (robust):             204.18
                            

In [ ]:
model = PanelOLS(panel3.fillna(0).Issues,
                 panel3.fillna(0).drop(columns = ['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6283
Estimator:                   PanelOLS   R-squared (Between):             -4.5495
No. Observations:                4000   R-squared (Within):               0.6285
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6283
Time:                        12:14:05   Log-likelihood                   -5446.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      279.93
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(24,3975)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             279.87
                            

## Table 4 model 3

In [ ]:
panel3 = panel_data[['Issues', 'Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3 = pd.concat([panel3.drop(columns= ['Issues_1',	'Issues_2',
                                          'Issues_3' ,1, 2, 3, 4]),
                    Sales_growth_1p, Sales_growth, Sales_growth_1,
             Sales_growth_2, Sales_growth_3, Sales_growth_4,
                    panel3[['Issues_1',	'Issues_2',
                           'Issues_3' ,1, 2, 3, 4]]], axis = 1)

panel3 

,,Issues,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,Sales_growth_1p,Sales_growth,Sales_growth_1,Sales_growth_2,Sales_growth_3,Sales_growth_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,1.498864,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,NaN,0.258709,0.048893,0.301701,-0.021183,0.238522,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,1.521491,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,NaN,NaN,NaN,NaN,NaN,NaN,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,1.481866,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,NaN,0.260948,0.146963,0.129414,-0.308850,0.065304,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna().Issues,
                 panel3.dropna().drop(columns = ['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6720
Estimator:                   PanelOLS   R-squared (Between):              0.2110
No. Observations:                2310   R-squared (Within):               0.6721
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6720
Time:                        12:14:05   Log-likelihood                   -3277.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      195.07
Entities:                          40   P-value                           0.0000
Avg Obs:                       57.750   Distribution:                 F(24,2285)
Min Obs:                       0.0000                                           
Max Obs:                       91.000   F-statistic (robust):             194.30
                            

In [ ]:
model = PanelOLS(panel3.fillna(0).Issues,
                 panel3.fillna(0).drop(columns = ['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6280
Estimator:                   PanelOLS   R-squared (Between):             -4.5873
No. Observations:                4000   R-squared (Within):               0.6282
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6280
Time:                        12:14:06   Log-likelihood                   -5448.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      279.55
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(24,3975)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             279.55
                            

## Table 4 model 4

In [ ]:
panel3 = panel_data[['Issues', 'Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3 = pd.concat([panel3.drop(columns= ['Issues_1',	'Issues_2',
                                          'Issues_3' ,1, 2, 3, 4]),
                    LEI_growth_1p, LEI_growth, LEI_growth_1,
             LEI_growth_2, LEI_growth_3, LEI_growth_4,
                    panel3[['Issues_1',	'Issues_2',
                           'Issues_3' ,1, 2, 3, 4]]], axis = 1)

panel3 

,,Issues,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,LEI_growth_1p,LEI_growth,LEI_growth_1,LEI_growth_2,LEI_growth_3,LEI_growth_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,1.498864,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,NaN,0.185510,0.194711,0.138602,0.131007,0.035842,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,1.521491,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,NaN,NaN,NaN,NaN,NaN,NaN,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,1.481866,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,NaN,0.078628,0.093646,0.137055,0.210666,0.388334,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna().Issues,
                 panel3.dropna().drop(columns = ['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6629
Estimator:                   PanelOLS   R-squared (Between):              0.4064
No. Observations:                2478   R-squared (Within):               0.6630
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6629
Time:                        12:14:06   Log-likelihood                   -3520.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      201.01
Entities:                          40   P-value                           0.0000
Avg Obs:                       61.950   Distribution:                 F(24,2453)
Min Obs:                       0.0000                                           
Max Obs:                       91.000   F-statistic (robust):             199.99
                            

In [ ]:
model = PanelOLS(panel3.fillna(0).Issues,
                 panel3.fillna(0).drop(columns = ['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6280
Estimator:                   PanelOLS   R-squared (Between):             -4.5826
No. Observations:                4000   R-squared (Within):               0.6283
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6280
Time:                        12:14:06   Log-likelihood                   -5448.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      279.65
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(24,3975)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             279.59
                            

## Table 4 model 5

In [ ]:
panel3 = panel_data[['Issues', 'Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3 = pd.concat([panel3.drop(columns= ['Issues_1',	'Issues_2',
                                          'Issues_3' ,1, 2, 3, 4]),
                    Idiosyncratic_volatility_1p, Idiosyncratic_volatility, Idiosyncratic_volatility_1,
             Idiosyncratic_volatility_2, Idiosyncratic_volatility_3, Idiosyncratic_volatility_4,
                    panel3[['Issues_1',	'Issues_2',
                           'Issues_3' ,1, 2, 3, 4]]], axis = 1)

panel3 

,,Issues,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,Idiosyncratic_volatility_1p,Idiosyncratic_volatility,Idiosyncratic_volatility_1,Idiosyncratic_volatility_2,Idiosyncratic_volatility_3,Idiosyncratic_volatility_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,1.498864,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,NaN,0.661048,0.359476,-0.676128,-0.090844,0.669536,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,1.521491,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,NaN,-0.271791,0.630312,-0.336260,1.243808,-0.940376,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,1.481866,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,NaN,0.903101,0.074506,-0.690757,0.483866,-0.320673,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna().Issues,
                 panel3.dropna().drop(columns = ['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6437
Estimator:                   PanelOLS   R-squared (Between):              0.1528
No. Observations:                3070   R-squared (Within):               0.6438
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6437
Time:                        12:14:06   Log-likelihood                   -4405.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      229.26
Entities:                          40   P-value                           0.0000
Avg Obs:                       76.750   Distribution:                 F(24,3045)
Min Obs:                       0.0000                                           
Max Obs:                       92.000   F-statistic (robust):             228.45
                            

In [ ]:
model = PanelOLS(panel3.fillna(0).Issues,
                 panel3.fillna(0).drop(columns = ['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6288
Estimator:                   PanelOLS   R-squared (Between):             -4.6281
No. Observations:                4000   R-squared (Within):               0.6290
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6288
Time:                        12:14:07   Log-likelihood                   -5443.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      280.57
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(24,3975)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             280.50
                            

## Table 4 model 6

In [ ]:
panel3 = panel_data[['Issues', 'Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3 = pd.concat([panel3.drop(columns= ['Issues_1',	'Issues_2',
                                          'Issues_3' ,1, 2, 3, 4]),
                   Stock_price_synchronicity_1p, Stock_price_synchronicity, Stock_price_synchronicity_1,
             Stock_price_synchronicity_2, Stock_price_synchronicity_3, Stock_price_synchronicity_4,
                    panel3[['Issues_1',	'Issues_2',
                           'Issues_3' ,1, 2, 3, 4]]], axis = 1)

panel3 

,,Issues,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,Stock_price_synchronicity_1p,Stock_price_synchronicity,Stock_price_synchronicity_1,Stock_price_synchronicity_2,Stock_price_synchronicity_3,Stock_price_synchronicity_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,1.498864,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,NaN,0.028460,0.674769,-0.098922,0.313311,0.696050,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,1.521491,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,NaN,1.920876,0.165917,-1.381572,0.911751,-0.791695,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,1.481866,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,NaN,1.715789,-0.518716,-0.235588,-0.278888,2.193861,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna().Issues,
                 panel3.dropna().drop(columns = ['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6429
Estimator:                   PanelOLS   R-squared (Between):              0.1279
No. Observations:                3070   R-squared (Within):               0.6430
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6429
Time:                        12:14:07   Log-likelihood                   -4408.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      228.44
Entities:                          40   P-value                           0.0000
Avg Obs:                       76.750   Distribution:                 F(24,3045)
Min Obs:                       0.0000                                           
Max Obs:                       92.000   F-statistic (robust):             227.63
                            

In [ ]:
model = PanelOLS(panel3.fillna(0).Issues,
                 panel3.fillna(0).drop(columns = ['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6284
Estimator:                   PanelOLS   R-squared (Between):             -4.6079
No. Observations:                4000   R-squared (Within):               0.6287
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6284
Time:                        12:14:07   Log-likelihood                   -5446.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      280.09
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(24,3975)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             280.03
                            

## Table 4 model 7

In [ ]:
panel3 = panel_data[['Issues', 'Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3 = pd.concat([panel3.drop(columns= ['Issues_1',	'Issues_2',
                                          'Issues_3' ,1, 2, 3, 4]),
                    GDP_growth_1p, GDP_growth, GDP_growth_1,
             GDP_growth_2, GDP_growth_3, GDP_growth_4,
             Sales_growth_1p, Sales_growth, Sales_growth_1,
             Sales_growth_2, Sales_growth_3, Sales_growth_4,
             
             LEI_growth_1p, LEI_growth, LEI_growth_1,
             LEI_growth_2, LEI_growth_3, LEI_growth_4,
             
             Idiosyncratic_volatility_1p, Idiosyncratic_volatility, Idiosyncratic_volatility_1,
             Idiosyncratic_volatility_2, Idiosyncratic_volatility_3, Idiosyncratic_volatility_4,
             
             Stock_price_synchronicity_1p, Stock_price_synchronicity, Stock_price_synchronicity_1,
             Stock_price_synchronicity_2, Stock_price_synchronicity_3, Stock_price_synchronicity_4,


                    panel3[['Issues_1',	'Issues_2',
                           'Issues_3' ,1, 2, 3, 4]]], axis = 1)

panel3 

,,Issues,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,GDP_growth_1p,GDP_growth,GDP_growth_1,GDP_growth_2,GDP_growth_3,GDP_growth_4,Sales_growth_1p,Sales_growth,Sales_growth_1,Sales_growth_2,Sales_growth_3,Sales_growth_4,LEI_growth_1p,LEI_growth,LEI_growth_1,LEI_growth_2,LEI_growth_3,LEI_growth_4,Idiosyncratic_volatility_1p,Idiosyncratic_volatility,Idiosyncratic_volatility_1,Idiosyncratic_volatility_2,Idiosyncratic_volatility_3,Idiosyncratic_volatility_4,Stock_price_synchronicity_1p,Stock_price_synchronicity,Stock_price_synchronicity_1,Stock_price_synchronicity_2,Stock_price_synchronicity_3,Stock_price_synchronicity_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,-1.117565,0.258314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,1.388266,1.042828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,0.640015,1.435284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,1.498864,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,NaN,0.208137,0.136682,0.285338,-0.103249,0.113881,NaN,0.258709,0.048893,0.301701,-0.021183,0.238522,NaN,0.185510,0.194711,0.138602,0.131007,0.035842,NaN,0.661048,0.359476,-0.676128,-0.090844,0.669536,NaN,0.028460,0.674769,-0.098922,0.313311,0.696050,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,1.521491,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.271791,0.630312,-0.336260,1.243808,-0.940376,NaN,1.920876,0.165917,-1.381572,0.911751,-0.791695,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,1.481866,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,NaN,0.022434,0.261901,0.085678,-0.384667,0.010417,NaN,0.260948,0.146963,0.129414,-0.308850,0.065304,NaN,0.078628,0.093646,0.137055,0.210666,0.388334,NaN,0.903101,0.074506,-0.690757,0.483866,-0.320673,NaN,1.715789,-0.518716,-0.235588,-0.278888,2.193861,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna().Issues,
                 panel3.dropna().drop(columns = ['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6766
Estimator:                   PanelOLS   R-squared (Between):              0.2285
No. Observations:                2249   R-squared (Within):               0.6766
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6766
Time:                        12:14:08   Log-likelihood                   -3186.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      95.874
Entities:                          40   P-value                           0.0000
Avg Obs:                       56.225   Distribution:                 F(48,2200)
Min Obs:                       0.0000                                           
Max Obs:                       91.000   F-statistic (robust):             95.469
                            

In [ ]:
model = PanelOLS(panel3.fillna(0).Issues,
                 panel3.fillna(0).drop(columns = ['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6315
Estimator:                   PanelOLS   R-squared (Between):             -4.5581
No. Observations:                4000   R-squared (Within):               0.6318
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6315
Time:                        12:14:08   Log-likelihood                   -5429.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      141.07
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(48,3951)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             141.07
                            

# Table 5

## Upper tab, model 1, Full sample

In [ ]:
panel3 = panel_data[['Issues', 'Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3

,,Issues,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,1.498864,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,1.521491,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,1.481866,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna().Issues,
                 panel3.dropna().drop(columns = ['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6421
Estimator:                   PanelOLS   R-squared (Between):              0.3458
No. Observations:                3087   R-squared (Within):               0.6421
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6421
Time:                        12:14:08   Log-likelihood                   -4438.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      305.74
Entities:                          40   P-value                           0.0000
Avg Obs:                       77.175   Distribution:                 F(18,3068)
Min Obs:                       0.0000                                           
Max Obs:                       92.000   F-statistic (robust):             304.54
                            

In [ ]:
model = PanelOLS(panel3.fillna(0).Issues,
                 panel3.fillna(0).drop(columns = ['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6273
Estimator:                   PanelOLS   R-squared (Between):             -4.6129
No. Observations:                4000   R-squared (Within):               0.6276
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6273
Time:                        12:14:08   Log-likelihood                   -5451.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      372.31
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(18,3981)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             372.22
                            

## Upper tab, model 2, $ROA > 0$

In [ ]:
# Defining ROAp, and its lags
ROAp = pd.read_excel(path, sheet_name='Issues ROA>0', skiprows=5)
ROAp = ROAp.set_index(['YearQuarter'])
ROAp  = pd.DataFrame(ROAp.stack(dropna = False ), columns=['Issues ROA>0'])
ROAp.index = ROAp.index.set_names(['YearQuarter', 'Country'])
#ROAp = pd.DataFrame(index = ROAp.index, columns= list_of_variables)
ROAp = ROAp.reset_index().set_index(['Country', 'YearQuarter'])
ROAp



,,Issues ROA>0
Country,YearQuarter,
ARG,19901,NaN
AUS,19901,NaN
AUT,19901,NaN
BEL,19901,NaN
BRA,19901,NaN
...,...,...
SWE,20144,0.524806
THA,20144,2.305493
USANASDAQ,20144,0.192464


In [ ]:
panel3 = panel_data[['Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3 = pd.concat([ROAp, panel3], axis = 1)

panel3 

,,Issues ROA>0,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,0.524806,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,2.305493,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,0.192464,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna()['Issues ROA>0'],
                 panel3.dropna().drop(columns = ['Issues ROA>0']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:           Issues ROA>0   R-squared:                        0.1267
Estimator:                   PanelOLS   R-squared (Between):             -0.1855
No. Observations:                3096   R-squared (Within):               0.1267
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.1267
Time:                        12:14:09   Log-likelihood                   -4292.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      24.790
Entities:                          40   P-value                           0.0000
Avg Obs:                       77.400   Distribution:                 F(18,3077)
Min Obs:                       0.0000                                           
Max Obs:                       92.000   F-statistic (robust):             24.396
                            

In [ ]:
model = PanelOLS(panel3.fillna(0)['Issues ROA>0'],
                 panel3.fillna(0).drop(columns = ['Issues ROA>0']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:           Issues ROA>0   R-squared:                        0.1116
Estimator:                   PanelOLS   R-squared (Between):          -8.659e+28
No. Observations:                4000   R-squared (Within):               0.1116
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.1116
Time:                        12:14:09   Log-likelihood                   -5190.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      27.774
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(18,3981)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             27.517
                            

## Upper tab, model 3, $ROA < 0$

In [ ]:
# Defining ROAn, and its lags
ROAn = pd.read_excel(path, sheet_name='Issues ROA<0', skiprows=5)
ROAn = ROAn.set_index(['YearQuarter'])
ROAn  = pd.DataFrame(ROAn.stack(dropna = False ), columns=['Issues ROA<0'])
ROAn.index = ROAn.index.set_names(['YearQuarter', 'Country'])
#ROAn = pd.DataFrame(index = ROAn.index, columns= list_of_variables)
ROAn = ROAn.reset_index().set_index(['Country', 'YearQuarter'])
ROAn



,,Issues ROA<0
Country,YearQuarter,
ARG,19901,NaN
AUS,19901,NaN
AUT,19901,NaN
BEL,19901,NaN
BRA,19901,NaN
...,...,...
SWE,20144,-0.390022
THA,20144,-1.344931
USANASDAQ,20144,0.825653


In [ ]:
panel3 = panel_data[['Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3 = pd.concat([ROAn, panel3], axis = 1)

panel3 

,,Issues ROA<0,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,-0.390022,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,-1.344931,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,0.825653,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna()['Issues ROA<0'],
                 panel3.dropna().drop(columns = ['Issues ROA<0']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:           Issues ROA<0   R-squared:                        0.0544
Estimator:                   PanelOLS   R-squared (Between):             -0.1952
No. Observations:                3096   R-squared (Within):               0.0544
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.0544
Time:                        12:14:10   Log-likelihood                   -4464.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      9.8352
Entities:                          40   P-value                           0.0000
Avg Obs:                       77.400   Distribution:                 F(18,3077)
Min Obs:                       0.0000                                           
Max Obs:                       92.000   F-statistic (robust):             9.6049
                            

In [ ]:
model = PanelOLS(panel3.fillna(0)['Issues ROA<0'],
                 panel3.fillna(0).drop(columns = ['Issues ROA<0']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:           Issues ROA<0   R-squared:                        0.0465
Estimator:                   PanelOLS   R-squared (Between):          -5.155e+28
No. Observations:                4000   R-squared (Within):               0.0466
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.0465
Time:                        12:14:10   Log-likelihood                   -5332.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      10.796
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(18,3981)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             10.528
                            

## Lower tab, model 1, Full sample

In [ ]:
panel3 = panel_data[['Issues', 'Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3

,,Issues,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,1.498864,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,1.521491,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,1.481866,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna().Issues,
                 panel3.dropna().drop(columns = ['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6421
Estimator:                   PanelOLS   R-squared (Between):              0.3458
No. Observations:                3087   R-squared (Within):               0.6421
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6421
Time:                        12:14:11   Log-likelihood                   -4438.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      305.74
Entities:                          40   P-value                           0.0000
Avg Obs:                       77.175   Distribution:                 F(18,3068)
Min Obs:                       0.0000                                           
Max Obs:                       92.000   F-statistic (robust):             304.54
                            

In [ ]:
model = PanelOLS(panel3.fillna(0).Issues,
                 panel3.fillna(0).drop(columns = ['Issues']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6273
Estimator:                   PanelOLS   R-squared (Between):             -4.6129
No. Observations:                4000   R-squared (Within):               0.6276
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6273
Time:                        12:14:11   Log-likelihood                   -5451.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      372.31
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(18,3981)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             372.22
                            

## Lower tab, model 2, $SEO$s

In [ ]:
# Defining SEOs, and its lags
SEOs = pd.read_excel(path, sheet_name='SEOs', skiprows=5)
SEOs = SEOs.set_index(['YearQuarter'])
SEOs  = pd.DataFrame(SEOs.stack(dropna = False ), columns=['SEOs'])
SEOs.index = SEOs.index.set_names(['YearQuarter', 'Country'])
#SEOs = pd.DataFrame(index = SEOs.index, columns= list_of_variables)
SEOs = SEOs.reset_index().set_index(['Country', 'YearQuarter'])
SEOs



,,SEOs
Country,YearQuarter,
ARG,19901,NaN
AUS,19901,NaN
AUT,19901,NaN
BEL,19901,NaN
BRA,19901,NaN
...,...,...
SWE,20144,0.210321
THA,20144,0.461566
USANASDAQ,20144,0.516548


In [ ]:
panel3 = panel_data[['Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3 = pd.concat([SEOs, panel3], axis = 1)

panel3 

,,SEOs,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,0.210321,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,0.461566,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,0.516548,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna()['SEOs'],
                 panel3.dropna().drop(columns = ['SEOs']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   SEOs   R-squared:                        0.1258
Estimator:                   PanelOLS   R-squared (Between):             -0.1335
No. Observations:                3031   R-squared (Within):               0.1258
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.1258
Time:                        12:14:12   Log-likelihood                   -4135.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      24.076
Entities:                          40   P-value                           0.0000
Avg Obs:                       75.775   Distribution:                 F(18,3012)
Min Obs:                       0.0000                                           
Max Obs:                       92.000   F-statistic (robust):             23.949
                            

In [ ]:
model = PanelOLS(panel3.fillna(0)['SEOs'],
                 panel3.fillna(0).drop(columns = ['SEOs']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   SEOs   R-squared:                        0.1179
Estimator:                   PanelOLS   R-squared (Between):          -1.444e+29
No. Observations:                4000   R-squared (Within):               0.1180
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.1179
Time:                        12:14:12   Log-likelihood                   -5034.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      29.570
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(18,3981)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             29.548
                            

## Lower tab, model 4, $IPO$s

In [ ]:
# Defining IPOs, and its lags
IPOs = pd.read_excel(path, sheet_name='IPOs', skiprows=5)
IPOs = IPOs.set_index(['YearQuarter'])
IPOs  = pd.DataFrame(IPOs.stack(dropna = False ), columns=['IPOs'])
IPOs.index = IPOs.index.set_names(['YearQuarter', 'Country'])
#IPOs = pd.DataFrame(index = IPOs.index, columns= list_of_variables)
IPOs = IPOs.reset_index().set_index(['Country', 'YearQuarter'])
IPOs



,,IPOs
Country,YearQuarter,
ARG,19901,NaN
AUS,19901,NaN
AUT,19901,NaN
BEL,19901,NaN
BRA,19901,NaN
...,...,...
SWE,20144,NaN
THA,20144,NaN
USANASDAQ,20144,0.183248


In [ ]:
panel3 = panel_data[['Liquidity_1p', 'Liquidity', 'Liquidity_1', 'Liquidity_2',
            'Liquidity_3', 'Liquidity_4', 'Market_returns_1p',
             'Market_returns',	'Market_returns_1',	'Market_returns_2',
            	'Market_returns_3',	'Market_returns_4',	'Issues_1',	'Issues_2',	'Issues_3', 1, 2, 3, 4]]

panel3 = pd.concat([IPOs, panel3], axis = 1)

panel3 

,,IPOs,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,Issues_1,Issues_2,Issues_3,1,2,3,4
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,NaN,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,-2.060938,0.749432,-0.187358,0,0,0,1
THA,20144,NaN,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,-1.217193,1.065044,-0.456447,0,0,0,1
USANASDAQ,20144,0.183248,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,-0.281044,-1.890656,1.379668,0,0,0,1


In [ ]:
model = PanelOLS(panel3.dropna()['IPOs'],
                 panel3.dropna().drop(columns = ['IPOs']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   IPOs   R-squared:                        0.1635
Estimator:                   PanelOLS   R-squared (Between):              0.1222
No. Observations:                2835   R-squared (Within):               0.1636
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.1635
Time:                        12:14:13   Log-likelihood                   -3733.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      30.580
Entities:                          40   P-value                           0.0000
Avg Obs:                       70.875   Distribution:                 F(18,2816)
Min Obs:                       0.0000                                           
Max Obs:                       92.000   F-statistic (robust):             30.358
                            

In [ ]:
model = PanelOLS(panel3.fillna(0)['IPOs'],
                 panel3.fillna(0).drop(columns = ['IPOs']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   IPOs   R-squared:                        0.1261
Estimator:                   PanelOLS   R-squared (Between):          -8.266e+28
No. Observations:                4000   R-squared (Within):               0.1261
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.1261
Time:                        12:14:13   Log-likelihood                   -4853.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      31.911
Entities:                          40   P-value                           0.0000
Avg Obs:                      100.000   Distribution:                 F(18,3981)
Min Obs:                      100.000                                           
Max Obs:                      100.000   F-statistic (robust):             31.843
                            

# Extension

In [ ]:
Issues.columns

Index(['ARG', 'AUS', 'AUT', 'BEL', 'BRA', 'CAN', 'CHE', 'CHL', 'CHN', 'COL',
       'DEU', 'DNK', 'EGY', 'ESP', 'FIN', 'FRA', 'GBR', 'GRC', 'HKG', 'IDN',
       'IND', 'ISR', 'ITA', 'JPN', 'KOR', 'MEX', 'MYS', 'NLD', 'NOR', 'NZL',
       'PHL', 'POL', 'PRT', 'RUS', 'SGP', 'SWE', 'THA', 'USANASDAQ', 'USANYSE',
       'ZAF'],
      dtype='object')

### table 2 model 1 for different countries

In [ ]:
Asian_countries = ['CHN', 'HKG', 'IDN', 'IND', 
                   'ISR', 'JPN', 'KOR', 'KOR', 'MYS', 'RUS', 'SGP', 'THA', 'PHL']

Europian_countries = ['AUT', 'BEL', 'CHE', 'DNK', 'ESP','FIN', 'FRA', 'GBR',
                      'GRC', 'ITA', 'DEU', 'NLD', 'NOR', 'POL', 'PRT', 'SWE']

north_american_countries = ['CAN', 'USANASDAQ', 'USANYSE']
south_american_countries = ['ARG', 'BRA', 'COL', 'MEX', 'CHL', ]
Australia_Newzland = ['AUS', 'NZL',]
African_countries = ['EGY', 'ZAF']

list_name = ['Asian_countries', 'Europian_countries', 'north_american_countries', 
               'south_american_countries', 'Australia_Newzland', 'African_countries']

In [ ]:
jj = 0
for region in [Asian_countries, Europian_countries, north_american_countries, 
               south_american_countries, Australia_Newzland, African_countries]:


  print('=================================================')
  print('result for ', list_name[jj])
  jj += 1
  print('=================================================')

  path = '/content/drive/MyDrive/PhD/ECF/Hanselaar_Stulz_vanDijk_data/Hanselaar_Stulz_vanDijk_data.xlsx'
  Issues = pd.read_excel(path, sheet_name='Issues', skiprows=5)
  Issues = Issues.set_index(['YearQuarter']) 
  Issues = Issues[region]

  Issues = Issues.diff()
  Issues_1 = Issues.shift()
  Issues_2 = Issues.shift(2)
  Issues_3 = Issues.shift(3)
  #Issues['Quarter'] = Issues.YearQuarter % 10 

  Liquidity = pd.read_excel(path, sheet_name='Liquidity', skiprows=5)
  Liquidity = Liquidity.set_index(['YearQuarter'])  
  Liquidity = Liquidity[region]

  Liquidity = Liquidity.diff()
  Liquidity_2p = Liquidity.shift(-2)
  Liquidity_1p = Liquidity.shift(-1)
  Liquidity_1 = Liquidity.shift(1)
  Liquidity_2 = Liquidity.shift(2)
  Liquidity_3 = Liquidity.shift(3)
  Liquidity_4 = Liquidity.shift(4)
  Liquidity_5 = Liquidity.shift(5)
  Liquidity_6 = Liquidity.shift(6)

 
  Market_returns = pd.read_excel(path, sheet_name='Market returns', skiprows=5)
  Market_returns = Market_returns.set_index(['YearQuarter'])  
  Market_returns = Market_returns[region]

  Market_returns_1p = Market_returns.shift(-1)
  Market_returns_1 = Market_returns.shift(1)
  Market_returns_2 = Market_returns.shift(2)
  Market_returns_3 = Market_returns.shift(3)
  Market_returns_4 = Market_returns.shift(4)





  names_of_variables = [Issues, 
                        Liquidity_2p, Liquidity_1p, Liquidity,
                        Liquidity_1, Liquidity_2, Liquidity_3, Liquidity_4,
                        Liquidity_5, Liquidity_6,
                        Market_returns_1p, Market_returns, Market_returns_1,
                        Market_returns_2, Market_returns_3, Market_returns_4,
                        Issues_1, Issues_2, Issues_3]

  list_of_variables = ['Issues', 
                        'Liquidity_2p', 'Liquidity_1p', 'Liquidity',
                        'Liquidity_1', 'Liquidity_2', 'Liquidity_3', 'Liquidity_4',
                        'Liquidity_5', 'Liquidity_6',
                        'Market_returns_1p', 'Market_returns', 'Market_returns_1',
                        'Market_returns_2', 'Market_returns_3', 'Market_returns_4',
                        'Issues_1', 'Issues_2', 'Issues_3']

  i = 0
  for data in names_of_variables:
    data  = pd.DataFrame(data.stack(dropna = False ), columns=[list_of_variables[i]])
    data.index = data.index.set_names(['YearQuarter', 'Country'])
    if i == 0: new_name_of_variables = pd.DataFrame(index = data.index, columns= list_of_variables)
    new_name_of_variables[list_of_variables[i]] = data
    i += 1


  panel_data = pd.concat([new_name_of_variables,
                          pd.get_dummies(pd.DataFrame(new_name_of_variables.index.get_level_values(0) % 10, 
                                                  index = new_name_of_variables.index)['YearQuarter'])], axis = 1)

  panel_data = panel_data.reset_index().set_index(['Country', 'YearQuarter'])


  model = PanelOLS(panel_data.dropna().Issues,
                  panel_data.dropna().drop(columns=['Issues', 'Liquidity_2p',
                                                    'Liquidity_5', 'Liquidity_6', 'Issues_2', 'Issues_3']))
  res = model.fit()

  print(res.summary)

result for  Asian_countries
                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.3722
Estimator:                   PanelOLS   R-squared (Between):             -1.5804
No. Observations:                 984   R-squared (Within):               0.3724
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.3722
Time:                        12:14:15   Log-likelihood                   -1628.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      35.835
Entities:                          12   P-value                           0.0000
Avg Obs:                       82.000   Distribution:                  F(16,967)
Min Obs:                       0.0000                                           
Max Obs:                       178.00   F-statistic (robust):             35.814


## Adding variables

In [ ]:
path = '/content/drive/MyDrive/PhD/ECF/Hanselaar_Stulz_vanDijk_data/Hanselaar_Stulz_vanDijk_data.xlsx'
Issues = pd.read_excel(path, sheet_name='Issues', skiprows=5)
Issues = Issues.set_index(['YearQuarter']) 
Issues = Issues.diff()
Issues_1 = Issues.shift()
Issues_2 = Issues.shift(2)
Issues_3 = Issues.shift(3)

Liquidity = pd.read_excel(path, sheet_name='Liquidity', skiprows=5)
Liquidity = Liquidity.set_index(['YearQuarter'])  
Liquidity = Liquidity.diff()
Liquidity_2p = Liquidity.shift(-2)
Liquidity_1p = Liquidity.shift(-1)
Liquidity_1 = Liquidity.shift(1)
Liquidity_2 = Liquidity.shift(2)
Liquidity_3 = Liquidity.shift(3)
Liquidity_4 = Liquidity.shift(4)
Liquidity_5 = Liquidity.shift(5)
Liquidity_6 = Liquidity.shift(6)


Market_returns = pd.read_excel(path, sheet_name='Market returns', skiprows=5)
Market_returns = Market_returns.set_index(['YearQuarter'])  

Market_returns_1p = Market_returns.shift(-1)
Market_returns_1 = Market_returns.shift(1)
Market_returns_2 = Market_returns.shift(2)
Market_returns_3 = Market_returns.shift(3)
Market_returns_4 = Market_returns.shift(4)



names_of_variables = [Issues, 
                      Liquidity_2p, Liquidity_1p, Liquidity, Liquidity ** 2,
                      Liquidity_1, Liquidity_2, Liquidity_3, Liquidity_4,
                      Liquidity_5, Liquidity_6,
                      Market_returns_1p, Market_returns, Market_returns ** 2, Market_returns_1,
                      Market_returns_2, Market_returns_3, Market_returns_4,
                      Issues_1, Issues_2, Issues_3]

list_of_variables = ['Issues', 
                      'Liquidity_2p', 'Liquidity_1p', 'Liquidity', 'Liquidity2', 
                      'Liquidity_1', 'Liquidity_2', 'Liquidity_3', 'Liquidity_4',
                      'Liquidity_5', 'Liquidity_6',
                      'Market_returns_1p', 'Market_returns', 'Market_returns2', 'Market_returns_1',
                      'Market_returns_2', 'Market_returns_3', 'Market_returns_4',
                      'Issues_1', 'Issues_2', 'Issues_3']

i = 0
for data in names_of_variables:
  data  = pd.DataFrame(data.stack(dropna = False ), columns=[list_of_variables[i]])
  data.index = data.index.set_names(['YearQuarter', 'Country'])
  if i == 0: new_name_of_variables = pd.DataFrame(index = data.index, columns= list_of_variables)
  new_name_of_variables[list_of_variables[i]] = data
  i += 1

panel_data = pd.concat([new_name_of_variables,
                        pd.get_dummies(pd.DataFrame(new_name_of_variables.index.get_level_values(0) % 10, 
                                                index = new_name_of_variables.index)['YearQuarter'])], axis = 1)



panel_data                                                

Issues  Liquidity_2p  Liquidity_1p  ...  2  3  4
YearQuarter Country                                          ...         
19901       ARG             NaN           NaN           NaN  ...  0  0  0
            AUS             NaN           NaN           NaN  ...  0  0  0
            AUT             NaN           NaN           NaN  ...  0  0  0
            BEL             NaN           NaN           NaN  ...  0  0  0
            BRA             NaN           NaN           NaN  ...  0  0  0
...                         ...           ...           ...  ... .. .. ..
20144       SWE        1.498864           NaN           NaN  ...  0  0  1
            THA        1.521491           NaN           NaN  ...  0  0  1
            USANASDAQ  1.481866           NaN           NaN  ...  0  0  1
            USANYSE    0.079183           NaN           NaN  ...  0  0  1
            ZAF        2.606112           NaN           NaN  ...  0  0  1

[4000 rows x 25 columns]

In [ ]:
panel_data = panel_data.reset_index().set_index(['Country', 'YearQuarter'])


model = PanelOLS(panel_data.dropna().Issues,
                panel_data.dropna().drop(columns=['Issues', 'Liquidity_2p',
                                                  'Liquidity_5', 'Liquidity_6', 'Issues_2', 'Issues_3']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.4751
Estimator:                   PanelOLS   R-squared (Between):             -0.3917
No. Observations:                3028   R-squared (Within):               0.4752
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.4751
Time:                        12:14:28   Log-likelihood                   -4942.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      151.33
Entities:                          40   P-value                           0.0000
Avg Obs:                       75.700   Distribution:                 F(18,3009)
Min Obs:                       0.0000                                           
Max Obs:                       89.000   F-statistic (robust):             151.17
                            

In [ ]:
panel_data = panel_data.reset_index().set_index(['Country', 'YearQuarter'])


model = PanelOLS(panel_data.dropna().Issues,
                panel_data.dropna().drop(columns=['Issues', 'Liquidity_2p',
                                                  'Liquidity_5', 'Liquidity_6']))
res = model.fit()

print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.6428
Estimator:                   PanelOLS   R-squared (Between):             -0.2510
No. Observations:                3028   R-squared (Within):               0.6429
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.6428
Time:                        12:14:28   Log-likelihood                   -4359.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      270.51
Entities:                          40   P-value                           0.0000
Avg Obs:                       75.700   Distribution:                 F(20,3007)
Min Obs:                       0.0000                                           
Max Obs:                       89.000   F-statistic (robust):             270.29
                            

## Cross sectional or time series regressions

In [ ]:
path = '/content/drive/MyDrive/PhD/ECF/Hanselaar_Stulz_vanDijk_data/Hanselaar_Stulz_vanDijk_data.xlsx'
Issues = pd.read_excel(path, sheet_name='Issues', skiprows=5)
Issues = Issues.set_index(['YearQuarter']) 
Issues = Issues.diff()
Issues_1 = Issues.shift()
Issues_2 = Issues.shift(2)
Issues_3 = Issues.shift(3)
#Issues['Quarter'] = Issues.YearQuarter % 10 

Liquidity = pd.read_excel(path, sheet_name='Liquidity', skiprows=5)
Liquidity = Liquidity.set_index(['YearQuarter'])  
Liquidity = Liquidity.diff()
Liquidity_2p = Liquidity.shift(-2)
Liquidity_1p = Liquidity.shift(-1)
Liquidity_1 = Liquidity.shift(1)
Liquidity_2 = Liquidity.shift(2)
Liquidity_3 = Liquidity.shift(3)
Liquidity_4 = Liquidity.shift(4)
Liquidity_5 = Liquidity.shift(5)
Liquidity_6 = Liquidity.shift(6)

#Liquidity['Quarter'] = Liquidity.YearQuarter % 10 

Market_returns = pd.read_excel(path, sheet_name='Market returns', skiprows=5)
Market_returns = Market_returns.set_index(['YearQuarter'])  

Market_returns_1p = Market_returns.shift(-1)
Market_returns_1 = Market_returns.shift(1)
Market_returns_2 = Market_returns.shift(2)
Market_returns_3 = Market_returns.shift(3)
Market_returns_4 = Market_returns.shift(4)

names_of_variables = [Issues, 
                      Liquidity_2p, Liquidity_1p, Liquidity,
                      Liquidity_1, Liquidity_2, Liquidity_3, Liquidity_4,
                      Liquidity_5, Liquidity_6,
                      Market_returns_1p, Market_returns, Market_returns_1,
                      Market_returns_2, Market_returns_3, Market_returns_4,
                      Issues_1, Issues_2, Issues_3]

list_of_variables = ['Issues', 
                      'Liquidity_2p', 'Liquidity_1p', 'Liquidity',
                      'Liquidity_1', 'Liquidity_2', 'Liquidity_3', 'Liquidity_4',
                      'Liquidity_5', 'Liquidity_6',
                      'Market_returns_1p', 'Market_returns', 'Market_returns_1',
                      'Market_returns_2', 'Market_returns_3', 'Market_returns_4',
                      'Issues_1', 'Issues_2', 'Issues_3']

i = 0
for data in names_of_variables:
  data  = pd.DataFrame(data.stack(dropna = False ), columns=[list_of_variables[i]])
  data.index = data.index.set_names(['YearQuarter', 'Country'])
  if i == 0: new_name_of_variables = pd.DataFrame(index = data.index, columns= list_of_variables)
  new_name_of_variables[list_of_variables[i]] = data
  i += 1


panel_data = pd.concat([new_name_of_variables,
                        pd.get_dummies(pd.DataFrame(new_name_of_variables.index.get_level_values(0) % 10, 
                                                index = new_name_of_variables.index)['YearQuarter'])], axis = 1)





In [ ]:
columns = ['HKG', 'IDN', 'IND', 
            'ISR', 'JPN', 'KOR', 'KOR', 'MYS', 'RUS', 'SGP', 'THA', 'PHL',
            'AUT', 'BEL', 'CHE', 'DNK', 'ESP','FIN', 'FRA', 'GBR',
            'GRC', 'ITA', 'DEU', 'NLD', 'NOR', 'POL', 'PRT', 'SWE',
            'CAN', 'USANASDAQ', 'USANYSE', 'BRA', 'COL', 'MEX',
            'CHL','AUS', 'NZL', 'EGY', 'ZAF']


pvals = pd.DataFrame(index = panel_data.drop(columns=['Issues', 'Liquidity_2p',
                         'Liquidity_5', 'Liquidity_6']).columns, columns = columns)

coef = pd.DataFrame(index = panel_data.drop(columns=['Issues', 'Liquidity_2p',
                         'Liquidity_5', 'Liquidity_6']).columns, columns = columns)
for country in columns:

  my_data = panel_data[panel_data.index.get_level_values('Country') == country]
                                        
  model = sm.OLS(my_data.dropna().Issues,
                  my_data.dropna().drop(columns=['Issues', 'Liquidity_2p',
                                                    'Liquidity_5', 'Liquidity_6']))
  res = model.fit()
  pvals[country] = np.round(res.pvalues, decimals=3)
  coef[country] = np.round(res.params, decimals=2)

pvals.to_excel('pvals.xlsx')
coef.to_excel('coef.xlsx')


Controlling for regional effect

In [ ]:
path = '/content/drive/MyDrive/PhD/ECF/Hanselaar_Stulz_vanDijk_data/Hanselaar_Stulz_vanDijk_data.xlsx'
Issues = pd.read_excel(path, sheet_name='Issues', skiprows=5)
Issues = Issues.set_index(['YearQuarter']) 
Issues = Issues.diff()
Issues_1 = Issues.shift()
Issues_2 = Issues.shift(2)
Issues_3 = Issues.shift(3)

Liquidity = pd.read_excel(path, sheet_name='Liquidity', skiprows=5)
Liquidity = Liquidity.set_index(['YearQuarter'])  
Liquidity = Liquidity.diff()
Liquidity_2p = Liquidity.shift(-2)
Liquidity_1p = Liquidity.shift(-1)
Liquidity_1 = Liquidity.shift(1)
Liquidity_2 = Liquidity.shift(2)
Liquidity_3 = Liquidity.shift(3)
Liquidity_4 = Liquidity.shift(4)
Liquidity_5 = Liquidity.shift(5)
Liquidity_6 = Liquidity.shift(6)

Market_returns = pd.read_excel(path, sheet_name='Market returns', skiprows=5)
Market_returns = Market_returns.set_index(['YearQuarter'])  

Market_returns_1p = Market_returns.shift(-1)
Market_returns_1 = Market_returns.shift(1)
Market_returns_2 = Market_returns.shift(2)
Market_returns_3 = Market_returns.shift(3)
Market_returns_4 = Market_returns.shift(4)



names_of_variables = [Issues, 
                      Liquidity_2p, Liquidity_1p, Liquidity,
                      Liquidity_1, Liquidity_2, Liquidity_3, Liquidity_4,
                      Liquidity_5, Liquidity_6,
                      Market_returns_1p, Market_returns, Market_returns_1,
                      Market_returns_2, Market_returns_3, Market_returns_4,
                      Issues_1, Issues_2, Issues_3]

list_of_variables = ['Issues', 
                      'Liquidity_2p', 'Liquidity_1p', 'Liquidity',
                      'Liquidity_1', 'Liquidity_2', 'Liquidity_3', 'Liquidity_4',
                      'Liquidity_5', 'Liquidity_6',
                      'Market_returns_1p', 'Market_returns', 'Market_returns_1',
                      'Market_returns_2', 'Market_returns_3', 'Market_returns_4',
                      'Issues_1', 'Issues_2', 'Issues_3']

i = 0
for data in names_of_variables:
  data  = pd.DataFrame(data.stack(dropna = False ), columns=[list_of_variables[i]])
  data.index = data.index.set_names(['YearQuarter', 'Country'])
  if i == 0: new_name_of_variables = pd.DataFrame(index = data.index, columns= list_of_variables)
  new_name_of_variables[list_of_variables[i]] = data
  i += 1


panel_data = pd.concat([new_name_of_variables,
                        pd.get_dummies(pd.DataFrame(new_name_of_variables.index.get_level_values(0) % 10, 
                                                index = new_name_of_variables.index)['YearQuarter']),
                        pd.get_dummies(pd.DataFrame(new_name_of_variables.index.get_level_values(1), 
                        index = new_name_of_variables.index))], axis = 1)

panel_data = panel_data.reset_index().set_index(['Country', 'YearQuarter'])


panel_data                                          

,,Issues,Liquidity_2p,Liquidity_1p,Liquidity,Liquidity_1,Liquidity_2,Liquidity_3,Liquidity_4,Liquidity_5,Liquidity_6,Market_returns_1p,Market_returns,Market_returns_1,Market_returns_2,Market_returns_3,Market_returns_4,Issues_1,Issues_2,Issues_3,1,2,3,4,Country_ARG,Country_AUS,Country_AUT,Country_BEL,Country_BRA,Country_CAN,Country_CHE,Country_CHL,Country_CHN,Country_COL,Country_DEU,Country_DNK,Country_EGY,Country_ESP,Country_FIN,Country_FRA,Country_GBR,Country_GRC,Country_HKG,Country_IDN,Country_IND,Country_ISR,Country_ITA,Country_JPN,Country_KOR,Country_MEX,Country_MYS,Country_NLD,Country_NOR,Country_NZL,Country_PHL,Country_POL,Country_PRT,Country_RUS,Country_SGP,Country_SWE,Country_THA,Country_USANASDAQ,Country_USANYSE,Country_ZAF
Country,YearQuarter,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ARG,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
AUS,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.298746,-0.982127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
AUT,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.828191,2.715251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BEL,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.488985,-1.029592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BRA,19901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWE,20144,1.498864,NaN,NaN,0.195308,1.982315,2.332305,-3.802165,0.623541,-2.506717,1.779274,NaN,-0.073180,-0.321618,-0.015944,0.121168,0.426346,-2.060938,0.749432,-0.187358,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
THA,20144,1.521491,NaN,NaN,0.514388,-2.201574,0.869238,-0.687669,5.630684,-3.714794,-2.273362,NaN,-0.473159,0.250893,0.256441,0.294427,-0.702545,-1.217193,1.065044,-0.456447,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
USANASDAQ,20144,1.481866,NaN,NaN,-0.651659,0.505154,-0.390191,-0.003982,0.018999,-0.070041,-0.254677,NaN,0.160907,-0.158250,0.004467,-0.371240,0.410902,-0.281044,-1.890656,1.379668,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
panel_data = panel_data.dropna()
panel_data = panel_data.loc[:, (panel_data != 0).any(axis=0)]


model = PanelOLS(panel_data.Issues,
                 panel_data.drop(columns=['Issues', 'Liquidity_2p',
                 'Liquidity_5', 'Liquidity_6', 'Issues_2', 'Issues_3']),
                 check_rank = False)
res = model.fit()

print(res.summary)



                          PanelOLS Estimation Summary                           
Dep. Variable:                 Issues   R-squared:                        0.4748
Estimator:                   PanelOLS   R-squared (Between):              1.0000
No. Observations:                3028   R-squared (Within):               0.4748
Date:                Wed, Jul 28 2021   R-squared (Overall):              0.4748
Time:                        12:24:49   Log-likelihood                   -4943.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      48.871
Entities:                          40   P-value                           0.0000
Avg Obs:                       75.700   Distribution:                 F(55,2973)
Min Obs:                       0.0000                                           
Max Obs:                       89.000   F-statistic (robust):             71.583
                            

/usr/local/lib/python3.7/dist-packages/linearmodels/panel/results.py:85: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
